<a href="https://colab.research.google.com/github/max-ostapenko/api_examples/blob/main/examples/python_notebook/bigquery_vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authorize GCP access

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'max-ostapenko' # @param {type: "string"}
dataset_id = 'vector_search' # @param {type: "string"}
location_id = 'us' # @param {type: "string"}

from google.cloud import bigquery
bq_client = bigquery.Client(project=project_id)

!bq mk --project_id={project_id} --location {location_id} --dataset {dataset_id}

# Connect ml model

In [ ]:
connection_id = "vertex_ai-remote_functions-big_lake"

# Create BigQuery connection to Vertex AI
!bq mk --project_id={project_id} --location={location_id} \
    --connection_type=CLOUD_RESOURCE --connection {connection_id}

# Extract service account ID from the connection details
import subprocess, json
connection_details = json.loads(
    subprocess.check_output("bq show --project_id={project_id} --location={location_id} --format=json --connection {connection_id}".format(
        project_id=project_id,
        location_id=location_id,
        connection_id=connection_id
        ).split(" ")
    ).decode('utf-8')
)
service_account = connection_details["cloudResource"]["serviceAccountId"]

# Authorise 'Vertex AI User' role for connection service account
!gcloud projects add-iam-policy-binding {project_id} \
    --member='serviceAccount:{service_account}' --role='roles/aiplatform.user' > /dev/null

# Create BQ ML model for embedding
create_model_query = """
CREATE OR REPLACE MODEL `{project_id}.{dataset_id}.multimodalembedding`
REMOTE WITH CONNECTION `{project_id}.{location_id}.vertex_ai-remote_functions-big_lake`
OPTIONS (ENDPOINT = "textembedding-gecko@latest");
""".format(dataset_id=dataset_id, project_id=project_id, location_id=location_id)

!bq query --project_id={project_id} --use_legacy_sql=false '{create_model_query}'

# Get product names

In [ ]:
get_product_names_query = """
CREATE OR REPLACE TABLE
    `{dataset_id}.items_name_embedded` AS (
    SELECT
        *
    FROM
        ML.GENERATE_TEXT_EMBEDDING( MODEL `{dataset_id}.multimodalembedding`,
            (
            SELECT
                DISTINCT items.item_name AS content
            FROM
                `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131`
            CROSS JOIN
                UNNEST(items) AS items
            WHERE
                item_name IS NOT NULL
                AND item_name != "(not set)"
            ORDER BY
                item_name ASC ),
            STRUCT(TRUE AS flatten_json_output) ) );
""".format(dataset_id=dataset_id)

!bq query --project_id={project_id} --use_legacy_sql=false '{get_product_names_query}'

query_product_names = """
SELECT
    content, statistics, ml_embed_text_status, text_embedding
FROM
`{dataset_id}.items_name_embedded`
LIMIT 5
""".format(dataset_id=dataset_id)

!bq query --project_id={project_id} --use_legacy_sql=false '{query_product_names}'

# Search for neighbours

In [ ]:
search_query = "Something for my daughter" # @param {type: "string"}

select_neighbours_query = """
SELECT
    base.content AS base_content,
    distance
FROM
    VECTOR_SEARCH( TABLE {dataset_id}.items_name_embedded,
        "text_embedding",
        (
        SELECT
            content,
            text_embedding
        FROM
            ML.GENERATE_TEXT_EMBEDDING( MODEL `{dataset_id}.multimodalembedding`,
                (
                SELECT
                    "{search_query}" AS content),
                STRUCT(TRUE AS flatten_json_output) ) ),
        "text_embedding",
        top_k => 5 );
""".format(
    dataset_id=dataset_id,
    search_query=search_query
)

!bq query --project_id={project_id} --use_legacy_sql=false '{select_neighbours_query}'

Costs: https://cloud.google.com/vertex-ai/pricing#generative_ai_models

https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text-embedding

ML.GENERATE_IMAGE_EMBEDDING is not available yet